In [5]:
import json
import requests
from random import randint
import numpy as np


In [6]:
total = 50 #random number just to start loop
offset= 0

iteminfo = []
allsizes = ["38", "39", "40", "41", "42", "43", "44", "45"]


while offset < total:
    r = requests.get('https://api.mercadolibre.com/sites/MLA/search?seller_id=257128833&category=MLA22389&offset='+str(offset))
    data = r.json()
    offset = offset + len(data['results'])
    total = data['paging']['total']
    for item in data['results']:
        newitem = {}
        newitem['title'] = item['title']
        newitem['price'] = item['price']
        newitem['stock'] = item['available_quantity']
        itemreq = requests.get('https://api.mercadolibre.com/items/'+item['id']+'?attributes=attributes,pictures')
        itemattributes = itemreq.json()
        for attr in itemattributes['attributes']:
            if attr['id'] == 'BRAND':
                newitem['brand'] = attr['value_name']
            if attr['id'] == 'RECOMMENDED_SPORTS':
                newitem['category'] = attr['value_name']
        #get only the first picture
        picture_id = itemattributes['pictures'][0]['id']
        picrequest = requests.get('https://api.mercadolibre.com/pictures/'+picture_id)
        picdata = picrequest.json()
        for pic in picdata['variations']:
            if pic['size'] == '400x400':
                newitem['pictureurl'] = pic['url']
                break
        if 'category' not in newitem:
            newitem['category'] = "Urban" 
        if 'brand' not in newitem:
            newitem['brand'] = "Other" 
        newitem['sizes'] = np.random.choice(allsizes, randint(1,8), False)
        iteminfo.append(newitem)

        




for art in iteminfo:
    print(art['title'])
    print(art['price'])
    print(art['stock'])
    print(art['brand'])
    print(art['sizes'])
    print(art['pictureurl'])
    print(art['category']+'\n')

In [7]:
import pymysql

db = pymysql.connect("localhost","root","nico","bookstore" )

cursor = db.cursor()


for article in iteminfo:
    cursor.execute("INSERT INTO `article` (`title`, `stock`, `price`, `picture`) VALUES (%s, %s, %s, %s)", (article['title'], str(article['stock']), str(article['price']), str(article['pictureurl'])))
    articleid = cursor.lastrowid
    cursor.execute("INSERT INTO `brand` (`name`, `article_id`) VALUES (%s, %s)", (article['brand'], articleid))
    cursor.execute("INSERT INTO `category` (`name`, `article_id`) VALUES (%s, %s)", (article['category'], articleid))
    for size in article['sizes']:
        cursor.execute("INSERT INTO `size` (`value`, `article_id`) VALUES (%s, %s)", (size, articleid))
    

    


db.close()